In [1]:
%load_ext rpy2.ipython
%R library(dplyr); library(magrittr);library(ggplot2)


Attaching package: ‘dplyr’

The following object is masked from ‘package:stats’:

    filter

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



<StrVector - Python:0x10a3cc830 / R:0x1008f9bf0>
[str, str, str, ..., str, str, str]

In [2]:
%%R

format_codon_m<-function(codonm, codonw){
    codon_w<-read.table(codonw, header=TRUE, sep="\t", na.strings="*****")
    codonm<-codonm%>%scan%>%matrix(ncol=61, byrow=T)
    rownames(codonm)<-codon_w$title
    ccn<-read.table("./scripts/codonR/codonm_colnames.txt", sep="\t")
    colnames(codonm)<-ccn$V2
    return(codonm)
}

gptr<-function(codonlist, gene){
        anti<-read.table("anti_codon_table.txt", header=TRUE, sep="\t")%>%filter(exact==1)
        codon_prop=vector()    
        codons_tested=vector()
        gene$codon<-rownames(gene)
        colnames(gene)[1]<-"count"
        merge(anti, gene, by="codon")->gene
        gene%>%filter(exact==1)%>%group_by(aa)%>%summarise(aa_count=sum(count))->aa_sum

        for (codon in codonlist){
            if (codon %in% gene$codon) {
            codons_tested=c(codons_tested, codon)
            a=gene$aa[which(codon==gene$codon)]
            n=gene$count[which(codon==gene$codon)]     #the number of exact match codons in the gene
            total=aa_sum$aa_count[which(a==aa_sum$aa)] #the total number of codons for the specific amino acid
            codon_prop=c(codon_prop, n/total)
        }}
        return(cbind.data.frame(codons_tested, codon_prop))
      }

Gptr<-function(codonlist, codonm){
    anti<-read.table("anti_codon_table.txt", header=TRUE, sep="\t")%>%filter(exact==1)
    genome_codon_prop=vector()
    codons_tested=vector()
    genome<-data.frame(colSums(codonm))
    genome$codon<-rownames(genome)
    colnames(genome)[1]<-"count"

    merge(anti, genome, by="codon")->genome

    genome%>%group_by(aa)%>%summarise(aa_count=sum(count))->genome_aa_sum

    for (codon in codonlist){
        if (codon %in% genome$codon){
        codons_tested=c(codons_tested, codon)
        a=genome$aa[which(codon==genome$codon)]
        n=genome$count[which(codon==genome$codon)]     #the number of exact match codons in the gene
        total=genome_aa_sum$aa_count[which(a==genome_aa_sum$aa)] #the total number of codons for the specific amino acid
        genome_codon_prop=c(genome_codon_prop, n/total)
        }}
        output<-cbind.data.frame(codons_tested, genome_codon_prop)
        return(output)
        }


In [53]:
%%R
format_codon_m("./data/svm1/Msmeg155.m","./data/svm1/Msmeg155.w")->codonm
#read.table("./data/svm1/problematic_msmeg_genes.txt")->probs
#codonm$gene<-rownames(codonm)
head(codonm)

Read 408273 items
                          TTT TTC TTA TTG TCT TCC TCA TCG TAT TAC TGT TGC TGG
MSMEI_0001                  1  21   0   4   1   2   1  15   1   8   0   1   5
MSMEI_0002                  5   5   0   0   3   4   7   6   1   3   5   2   1
MSMEI_0003                  1  13   0   5   0   9   0   7   2   3   0   1   2
MSMEI_0004                  0   8   0   6   0   2   0  10   0   8   0   0   5
MSMEI_0005                  2   6   0   5   1   2   1  11   3   5   0   2   5
MSMEI_0006                  0   1   0   1   0   1   1   5   0   2   0   0   5
                          CTT CTC CTA CTG CCT CCC CCA CCG CAT CAC CAA CAG CGT
MSMEI_0001                  1  18   0  22   0  10   1  19   0   8   2  16   9
MSMEI_0002                  1   3   0   5   2   4   2   6   2   9   4   2   2
MSMEI_0003                  1  11   0  28   3   4   0  16   1   2   0   4   5
MSMEI_0004                  0   7   1  11   0   3   0  12   2   7   1   5   3
MSMEI_0005                  1  23   0  18   0 

In [54]:
%%R
problemcodonm<-codonm[probs$V1,]
head(problemcodonm)
codonlist<-read.table(paste("./data/svm1/JF937105.1.trna",sep=""), skip=3, header=FALSE,sep="\t")
codonlist$V6%>%unique->codonlist
codonlist

 [1] TGG AAC GCG CAG CCA TTC ATG CGT CAC TGC CTA AAG ATC GGA GTA ACT ??? GAC GAG
19 Levels: ??? AAC AAG ACT ATC ATG CAC CAG CCA CGT CTA GAC GAG GCG GGA ... TTC


In [62]:
%%R

print(Gene_rep)
print(Genome_rep)

   codons_tested codon_prop
1            TGG 1.00000000
2            AAC 1.00000000
3            GCG 0.46153846
4            CAG 0.88888889
5            CCA 0.03333333
6            TTC 0.95454545
7            ATG 1.00000000
8            CGT 0.20000000
9            CAC 1.00000000
10           TGC 1.00000000
11           CTA 0.00000000
12           AAG 0.90000000
13           ATC 0.96774194
14           GGA 0.09090909
15           GTA 0.00000000
16           ACT 0.06060606
17           GAC 0.76923077
18           GAG 0.81578947
   codons_tested genome_codon_prop
1            TGG        1.00000000
2            AAC        0.88624543
3            GCG        0.41698645
4            CAG        0.85958644
5            CCA        0.05523552
6            TTC        0.94891924
7            ATG        1.00000000
8            CGT        0.12613940
9            CAC        0.81353284
10           TGC        0.80738462
11           CTA        0.01022377
12           AAG        0.81953815
13           

In [69]:
%%R
for (i in seq(1,10)){
gptr(codonlist, data.frame(problemcodonm[i,]))->Gene_rep
    print(i)
    print(Gene_rep)
#Gptr(codonlist, data.frame(codonm))->Genome_rep
#Together<-merge(Genome_rep, Gene_rep, by="codons_tested")%>%mutate(pig=codon_prop/genome_codon_prop)
#ptcs<-mean(na.omit(Together$pig))
#print(Together)
#print(ptcs)
    }

[1] 1
   codons_tested codon_prop
1            TGG 1.00000000
2            AAC 1.00000000
3            GCG 0.46153846
4            CAG 0.88888889
5            CCA 0.03333333
6            TTC 0.95454545
7            ATG 1.00000000
8            CGT 0.20000000
9            CAC 1.00000000
10           TGC 1.00000000
11           CTA 0.00000000
12           AAG 0.90000000
13           ATC 0.96774194
14           GGA 0.09090909
15           GTA 0.00000000
16           ACT 0.06060606
17           GAC 0.76923077
18           GAG 0.81578947
[1] 2
   codons_tested codon_prop
1            TGG 1.00000000
2            AAC 0.80000000
3            GCG 0.33333333
4            CAG 0.33333333
5            CCA 0.14285714
6            TTC 0.50000000
7            ATG 1.00000000
8            CGT 0.08333333
9            CAC 0.81818182
10           TGC 0.28571429
11           CTA 0.00000000
12           AAG 0.00000000
13           ATC 0.57142857
14           GGA 0.60000000
15           GTA 0.00000000
16      

In [70]:
%%R
problemcodonm[4,]

TTT TTC TTA TTG TCT TCC TCA TCG TAT TAC TGT TGC TGG CTT CTC CTA CTG CCT CCC CCA 
  0   8   0   6   0   2   0  10   0   8   0   0   5   0   7   1  11   0   3   0 
CCG CAT CAC CAA CAG CGT CGC CGA CGG ATT ATC ATA ATG ACT ACC ACA ACG AAT AAC AAA 
 12   2   7   1   5   3   5   1   5   0  16   0   9   0   7   0   2   0   6   0 
AAG AGT AGC AGA AGG GTT GTC GTA GTG GCT GCC GCA GCG GAT GAC GAA GAG GGT GGC GGA 
  8   2   6   0   1   0  11   1  13   2  18   3  10   3  13   9  15  14  22   2 
GGG 
  1 


In [72]:
%%R
gene<-data.frame(problemcodonm[4,])
anti<-read.table("anti_codon_table.txt", header=TRUE, sep="\t")%>%filter(exact==1)
codon_prop=vector()    
codons_tested=vector()
gene$codon<-rownames(gene)
colnames(gene)[1]<-"count"
merge(anti, gene, by="codon")->gene
gene%>%filter(exact==1)%>%group_by(aa)%>%summarise(aa_count=sum(count))->aa_sum

for (codon in codonlist){
    if (codon %in% gene$codon) {
    codons_tested=c(codons_tested, codon)
    a=gene$aa[which(codon==gene$codon)]
    n=gene$count[which(codon==gene$codon)]     #the number of exact match codons in the gene
    total=aa_sum$aa_count[which(a==aa_sum$aa)] #the total number of codons for the specific amino acid
    print(total)
    print(n/total)
    codon_prop=c(codon_prop, n/total)
}}

[1] 5
[1] 1
[1] 6
[1] 1
[1] 33
[1] 0.3030303
[1] 6
[1] 0.8333333
[1] 15
[1] 0
[1] 8
[1] 1
[1] 9
[1] 1
[1] 15
[1] 0.2
[1] 9
[1] 0.7777778
[1] 0
[1] NaN
[1] 25
[1] 0.04
[1] 8
[1] 1
[1] 16
[1] 1
[1] 39
[1] 0.05128205
[1] 25
[1] 0.04
[1] 9
[1] 0
[1] 16
[1] 0.8125
[1] 24
[1] 0.625
